In [8]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam
# Используем препроцессинг именно для MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import numpy as np
from PIL import Image, ImageEnhance

# Параметры
batch_size = 16
img_size = (224, 224)
epochs = 100
data_path = "B:/train/croped_dataset_train"

# Проверка распределения классов
class_counts = {}
for class_name in os.listdir(data_path):
    class_path = os.path.join(data_path, class_name)
    if os.path.isdir(class_path):
        class_counts[class_name] = len(os.listdir(class_path))
print("Распределение классов:", class_counts)

# Генератор данных с аугментацией и корректной предобработкой для MobileNetV3Small
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Применяем препроцессинг MobileNetV3Small
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Было 'sparse', теперь 'categorical'
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # То же самое для валидации
    subset='validation',
    shuffle=False
)

# Количество классов
num_classes = len(train_generator.class_indices)

# Создание базовой модели MobileNetV3Small
base_model = MobileNetV3Small(include_top=False, input_shape=(img_size[0], img_size[1], 3), weights='imagenet')
base_model.trainable = True

# Замораживаем все слои, кроме верхних 50
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Построение модели
model = models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.GlobalAveragePooling2D(),
    # Оптимизированный Dropout (уменьшено с 0.8 до 0.5 для баланса регуляризации и обучения)
    layers.Dropout(0.5),
    # Добавляем L2-регуляризацию для борьбы с переобучением
    layers.Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01))
])

# Компиляция модели с использованием label smoothing (0.1)
loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(
    optimizer=Adam(learning_rate=2e-4),
    loss=loss_fn,
    metrics=['accuracy']
)

# Callbacks для оптимизации обучения
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint("B:/train/third/best_MobileNetV3Small.keras", monitor='val_loss', save_best_only=True, verbose=1)

callbacks = [reduce_lr, early_stop, checkpoint]

# Обучение модели
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Вывод словаря классов
class_indices = train_generator.class_indices
print("Class indices:", class_indices)

# Инвертируем словарь для отображения предсказанного класса по индексу
inverted_class_indices = {v: k for k, v in class_indices.items()}

# Сохранение финальной модели
model.save("B:/train/third/MobileNetV3Small_final.keras")

# Функция предобработки изображения для инференса
def preprocess_image(image_path):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Пример инференса: проход по изображениям из указанной папки
image_folder = "B:/train/croped_dataset_train/10_posuda_dlya_kumisa"
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

for image_path in image_files:
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = inverted_class_indices.get(predicted_class_index, "Unknown")
    print(f"Изображение: {image_path} | Предсказанный класс: {predicted_class}")


Распределение классов: {'0_verblyud_skulptura': 161, '10_posuda_dlya_kumisa': 305, '12_sandyk1': 143, '13_dombri': 252, '14_sedlo1': 143, '15_sedlo2': 123, '16_kobiz_vaza_gobelen': 167, '17_gobelen_ornament_alasha': 148, '18_ukrasheniya': 99, '19_fancy_vaza': 200, '1_barashek': 193, '20_yarkiy_gobelen_ledi': 195, '21_mujchina_s_mechom': 160, '22_jambyl': 149, '23_revolyuchiya_v_aule': 60, '24_dekorativniyi_vazi_vmeste': 237, '25_voylok_na_kruge_abstract': 192, '26_berkutchi_s_ptichey': 140, '27_turksib': 124, '28_shokan_sidit': 137, '29_vesna_jaylau_zagatovka_productov': 119, '2_tri_skulpturi_s_sharami': 303, '30_senokos': 138, '31_gori_i_voda': 55, '32_mujchina_s_gazetoy': 58, '33_medeu': 60, '34_ges': 60, '35_vitrina_s_ukrasheniyami': 60, '36_gobelen_derevo': 252, '37_gobelen_shahmati': 225, '38_gobelen_krasniy': 60, '39_loshadki_na_jaylau': 149, '3_chernaya_vaza': 162, '40_Abishev_klich': 112, '41_Aytbayev_schaste': 157, '42_Ahmetov_kompoziciya_4': 153, '43_Dosmogambetov_Suleymenov'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [15]:
# Проверка и балансировка классов
class_counts = {}
for class_name in os.listdir(data_path):
    class_path = os.path.join(data_path, class_name)
    if os.path.isdir(class_path):
        class_counts[class_name] = len(os.listdir(class_path))
print("Распределение классов:", class_counts)# Проход по всем изображениям в папке
image_folder = "B:/train/croped_dataset_train/8_cvetushaya_yablonya"  # Укажите путь к папке с изображениями
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

for image_path in image_files:
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class = inverted_class_indices.get(predicted_class_index, "Unknown")
    print(f"Изображение: {image_path} | Предсказанный класс: {predicted_class}")

Распределение классов: {'0_verblyud_skulptura': 161, '10_posuda_dlya_kumisa': 305, '12_sandyk1': 143, '13_dombri': 252, '14_sedlo1': 143, '15_sedlo2': 123, '16_kobiz_vaza_gobelen': 167, '17_gobelen_ornament_alasha': 148, '18_ukrasheniya': 99, '19_fancy_vaza': 200, '1_barashek': 193, '20_yarkiy_gobelen_ledi': 195, '21_mujchina_s_mechom': 160, '22_jambyl': 149, '23_revolyuchiya_v_aule': 60, '24_dekorativniyi_vazi_vmeste': 237, '25_voylok_na_kruge_abstract': 192, '26_berkutchi_s_ptichey': 140, '27_turksib': 124, '28_shokan_sidit': 137, '29_vesna_jaylau_zagatovka_productov': 119, '2_tri_skulpturi_s_sharami': 303, '30_senokos': 138, '31_gori_i_voda': 55, '32_mujchina_s_gazetoy': 58, '33_medeu': 60, '34_ges': 60, '35_vitrina_s_ukrasheniyami': 60, '36_gobelen_derevo': 252, '37_gobelen_shahmati': 225, '38_gobelen_krasniy': 60, '39_loshadki_na_jaylau': 149, '3_chernaya_vaza': 162, '40_Abishev_klich': 112, '41_Aytbayev_schaste': 157, '42_Ahmetov_kompoziciya_4': 153, '43_Dosmogambetov_Suleymenov'